In [ ]:
# from dask.distributed import Client
# import dask.dataframe as dd

# client = Client()
# client

In [25]:
import numpy as np

import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import logging
from dask.distributed import Client

import re

In [26]:
ddf = dd.read_csv("small_reviews.csv", blocksize=50e6, usecols=['reviewerID', 'overall','helpful','reviewTime'])

ddf.head()

,reviewerID,helpful,overall,reviewTime
0,A2T0RJ91B0PQ03,"[0, 0]",1.0,"05 21, 2014"
1,A3TYW0XA8HSGWB,"[0, 0]",5.0,"04 27, 2014"
2,A2CME0TQU2IVVB,"[1, 1]",5.0,"12 19, 2012"
3,A2E5IDLX7R388S,"[0, 0]",5.0,"06 10, 2014"
4,A3CIEMYUGV6ZMR,"[0, 0]",5.0,"04 11, 2012"


In [27]:


ddf = ddf.assign(reviewYear = lambda x: x.reviewTime.apply(
    lambda x: int(re.search(r"\d{4}", x).group()),
    meta=('reviewTime', 'str')))

def addHelpUnhelp(df):
    helpful_series = df['helpful'].apply(lambda x: np.array(re.findall(r"\d+", x)).astype(int))
    df[['newHelpful', 'totalreviews']] = pd.DataFrame(helpful_series.to_list())
    df['totalreviews'] = df['newHelpful'] + df['totalreviews']
    return df

metadata = {'reviewerID': 'str',
            'helpful': 'str',
            'overall': 'float',
            'reviewTime': 'str',
            'reviewYear': 'float',
            'newHelpful' : 'float',
            'totalreviews': 'float'
            }

ddf = ddf.map_partitions(addHelpUnhelp,
                   #.apply(
    ##lambda x: np.array(re.findall(r"\d+", x)).astype(int),
    meta=metadata
                  )

ddf.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [28]:
ddf.compute().head()

,reviewerID,helpful,overall,reviewTime,reviewYear,newHelpful,totalreviews
0,A2T0RJ91B0PQ03,"[0, 0]",1.0,"05 21, 2014",2014,0,0
1,A3TYW0XA8HSGWB,"[0, 0]",5.0,"04 27, 2014",2014,0,0
2,A2CME0TQU2IVVB,"[1, 1]",5.0,"12 19, 2012",2012,1,2
3,A2E5IDLX7R388S,"[0, 0]",5.0,"06 10, 2014",2014,0,0
4,A3CIEMYUGV6ZMR,"[0, 0]",5.0,"04 11, 2012",2012,0,0


In [29]:
#drop extra columnsTODO

#index??
#empiricaly check partitions
groups = ddf.groupby('reviewerID')

In [47]:
ret = groups.aggregate(
{
    'reviewTime' : 'count',
    'overall' : 'mean',
    'reviewYear' : 'min',
    'newHelpful' : 'sum',
    'totalreviews' : 'sum',
})

#split_out=4)

ret.rename(columns={#"reviewerID", 
                    "reviewTime":"number_products_rated", 
                    "overall":"avg_ratings", 
                    "reviewYear":"reviewing_since", 
                    "newHelpful":"helpful_votes", 
                    "totalreviews":"total_votes"})

ret.visualize()


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

In [48]:
ret.compute().head()

,reviewTime,overall,reviewYear,newHelpful,totalreviews
reviewerID,,,,,
A2T0RJ91B0PQ03,1,1.0,2014,0,0
A3TYW0XA8HSGWB,1,5.0,2014,0,0
A2CME0TQU2IVVB,1,5.0,2012,1,2
A2E5IDLX7R388S,1,5.0,2014,0,0
A3CIEMYUGV6ZMR,1,5.0,2012,0,0


In [50]:
ret['reviewerID']

KeyError: 'reviewerID'

In [ ]:
reviews = pd.read_csv("small_reviews.csv")

In [ ]:
number_products_rated = reviews.groupby('reviewerID').count()['asin']
avg_ratings = reviews.groupby('reviewerID')['overall'].mean()
reviewing_since = reviews.groupby('reviewerID')['reviewTime'].min()
helpful_reviews = reviews.groupby('reviewerID')['helpful'].agg(lambda x: sum([int(i.strip('[]').split(', ')[0]) for i in x]))
total_reviews = reviews.groupby('reviewerID')['helpful'].agg(lambda x: sum([int(i.strip('[]').split(', ')[0]) + int(i.strip('[]').split(', ')[1]) for i in x]))


In [ ]:

users = pd.DataFrame(index = set(reviews['reviewerID']))
users['number_products_rated'] = number_products_rated
users['avg_ratings'] = avg_ratings
users['reviewing_since'] = reviewing_since
users['helpful_reviews'] = helpful_reviews
users['total_reviews'] = total_reviews
users.reset_index(inplace=True)
users = users.rename(columns = {'index':'reviewerID (PRIMARY KEY)'})
users
